<h1>목표</h1>

- 학습 성능을 향상시키기 위한 전처리 방법 중 하나로써 CLAHE 영상 처리를 진행함.
- 학습 네트워크의 입력 데이터 형태에 맞게 데이터를 생성함.


<h4> CLAHE(Contrast Limited Adaptive Histogram Equalization): 대비 제한 적응 히스토그램 평활화 </h4>
    
- 영상의 대조도를 높이기 위한 전처리 적용
- Histogram equalization(HE): 각각의 값이 전체 분포에 차지하는 비중에 따라 분포를 재분배하므로 명암 대비를 개선하는 데 효과적

    <img src="fig_source/clahe.png"></img>
    - CLAHE: 이미지를 일정한 영역(tileGridSize 파라미터)으로 나누어 평탄화를 적용
    - 일정한 영역 내에서 극단적으로 어둡거나 밝은 부분이 있으면 노이즈가 발생
    - 어떤 영역이든 지정된 제한 값(clipLimit)을 넘으면 그 픽셀은 다른 영역에 균일하게 배분하여 적용

- CLAHE의 주요 매개변수: clipLimit와 tileGridSize
1. Clip Limit (clipLimit)
    - clipLimit는 히스토그램이 클립되는 수준을 결정하는 매개변수. 기본 AHE에서는 히스토그램 평활화가 매우 큰 대비를 초래하지만, CLAHE에서는 클립 한계를 설정하여 과도한 대비를 방지함.
    - 낮은 clipLimit: 대비가 적당히 개선됨. 노이즈가 덜 강화됨.
    - 높은 clipLimit: 대비가 크게 개선됨. 노이즈도 함께 강조될 수 있음.

2. Tile Grid Size (tileGridSize)
    - tileGridSize는 이미지가 타일로 나누어지는 크기를 결정하는 매개변수. CLAHE는 이미지를 작은 타일로 나누고, 각 타일에 대해 히스토그램 평활화를 수행한 후 타일 간의 경계를 부드럽게 연결함.

    - 작은 tileGridSize: 국소적인 대비가 크게 개선됨. 타일 경계가 눈에 띌 수 있음.
    - 큰 tileGridSize: 전체적인 대비 개선 효과가 줄어듦. 타일 경계가 덜 눈에 띔.

* 일반적인 유방 X선 영상에서의 병변의 대비 향상을 위하여 전처리로 CLAHE를 사용하며, 경험적인 수치로 선정함.
    - 한 연구에서 여러 변수의 비교 중 12.0의 ClipLimit와 8x8의 tilegrid를 적용했을 때 가장 높은 성능을 보임.
        - Jiang Z, Gandomkar Z, Trieu PD, Tavakoli Taba S, Barron ML, Obeidy P, Lewis SJ. Evaluating Recalibrating AI Models for Breast Cancer Diagnosis in a New Context: Insights from Transfer Learning, Image Enhancement and High-Quality Training Data Integration. Cancers. 2024; 16(2):322. https://doi.org/10.3390/cancers16020322
        => Australian mammographic database

In [ ]:
import os, glob
import SimpleITK as sitk
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
import tqdm

In [ ]:
# Getting data name
path_data = os.path.join('/home/user/workdir/notices/data', 'breast')

path_abimg = os.path.join(path_data, 'image', 'abnormal')
path_nrimg = os.path.join(path_data, 'image', 'normal')

abnameList = sorted(glob.glob(os.path.join(path_abimg, '*.dcm')))
nrnameList = sorted(glob.glob(os.path.join(path_nrimg, '*.dcm')))

nameList = abnameList+nrnameList
labList = [1 if n < len(abnameList) else 0 for n in range(len(nameList))]

print(f'Abnormal: {len(abnameList)}')
print(f'Normal: {len(nrnameList)}')

In [ ]:
# Data preprocessing in a sample data
sample_num = 5
abname = abnameList[sample_num]
print(f'Sample name: {abname}')

## Checking a original sample image
img = sitk.ReadImage(os.path.join(abname))
img = sitk.GetArrayFromImage(img)[0]

plt.imshow(img, cmap='gray')

In [ ]:
## OpenCV(cv2) library를 활용한 CLAHE 전처리 적용 방법
fixed_clip = 2.0
fixed_tile = 16
clahe = cv2.createCLAHE(clipLimit = fixed_clip, tileGridSize = (fixed_tile,fixed_tile))
img_clahe = clahe.apply(img)

In [ ]:
# Checking tileGridSize
# Apply CLAHE with different tile grid sizes
clahe_1 = cv2.createCLAHE(clipLimit=fixed_clip, tileGridSize=(2, 2)).apply(img)
clahe_2 = cv2.createCLAHE(clipLimit=fixed_clip, tileGridSize=(16, 16)).apply(img)
clahe_3 = cv2.createCLAHE(clipLimit=fixed_clip, tileGridSize=(100, 100)).apply(img)

# Plot results
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
axs[0].imshow(img, cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(clahe_1, cmap='gray')
axs[1].set_title('CLAHE tileGridSize=(2, 2)')
axs[2].imshow(clahe_2, cmap='gray')
axs[2].set_title('CLAHE tileGridSize=(16, 16)')
axs[3].imshow(clahe_3, cmap='gray')
axs[3].set_title('CLAHE tileGridSize=(100, 100)')
plt.savefig('clahe_comparison_fixedClipLimit.png')
plt.show()

In [ ]:
# Checking tileGridSize
# Apply CLAHE with different tile grid sizes
clahe_1 = cv2.createCLAHE(clipLimit=0.01, tileGridSize=(fixed_tile, fixed_tile)).apply(img)
clahe_2 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(fixed_tile, fixed_tile)).apply(img)
clahe_3 = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(fixed_tile, fixed_tile)).apply(img)

# Plot results
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
axs[0].imshow(img, cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(clahe_1, cmap='gray')
axs[1].set_title('CLAHE clipLimit=0.01')
axs[2].imshow(clahe_2, cmap='gray')
axs[2].set_title('CLAHE clipLimit=2.0')
axs[3].imshow(clahe_3, cmap='gray')
axs[3].set_title('CLAHE clipLimit=40.0')
plt.savefig('clahe_comparison_fixedTileGrid.png')
plt.show()

In [ ]:
# Saving data
## clahe 설정
clahe = cv2.createCLAHE(clipLimit = fixed_clip, tileGridSize = (fixed_tile,fixed_tile)) 

## 데이터 로드 및 clahe 적용 함수 설정
def getting_img(path_img, imageSize):
    # loading image from dicom format
    img = sitk.ReadImage(path_img)
    img = sitk.GetArrayFromImage(img)[0]
    
    img = clahe.apply(img)
    img = (img-img.min())/(img.max()-img.min())  # min-max normalization

    img = cv2.resize(img, (imageSize, imageSize), interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)   # expanding image dimension for stacking data
    
    return img

In [ ]:
# Saving grayscale images as numpy format

n_class = 2         # Normal: 0 | Abnormal: 1 
imageSize = 512     # 임의 설정값. 

os.makedirs(os.path.join(os.path.abspath('.'), 'npy'), exist_ok=True)           # Setting saving path


## Making empty array for stacking all data
array_img = np.ndarray((0,imageSize,imageSize), np.float32)
array_lab = np.ndarray((0,n_class))

## Loading and Stacking data
for name in tqdm.tqdm(nameList):

    img = getting_img(name, imageSize)   # Getting image data
    
    # Making label data
    if 'AN' in os.path.basename(name):   # abnormal label: 1
        lab = np.asarray([0, 1])
    else:
        lab = np.asarray([1, 0])

    array_img = np.concatenate([array_img, img], axis=0)    # Stacking data
    array_lab = np.concatenate([array_lab, np.expand_dims(lab, axis=0)], axis=0)

In [ ]:
print(f'Image shape: {array_img.shape} | Label shape: {array_lab.shape}')

In [ ]:
# Check data shape
array_img = np.expand_dims(array_img, axis=-1)      # input shape에 맞춰줌. (n_images, image_height, image_width, channel)
print(f'Image shape: {array_img.shape} | Label shape: {array_lab.shape}')

In [ ]:
## Saving data for numpy format
### => 학습에 활용될 데이터 미리 저장
### => 학습용 데이터: Train, 성능 검증용 데이터 Test
train_x, test_x, train_y, test_y= train_test_split(array_img, array_lab, test_size=0.1, random_state=4, stratify= array_lab)

path_save = os.path.join(os.path.abspath('.'), 'npy')
os.makedirs(path_save, exist_ok=True)

np.save(os.path.join(path_save, 'train_img.npy'), train_x)
np.save(os.path.join(path_save, 'test_img.npy'), test_x)
np.save(os.path.join(path_save, 'train_lab.npy'), train_y)
np.save(os.path.join(path_save, 'test_lab.npy'), test_y)
